In [1]:
library(MASS)
library(dplyr)
library(tidyr)
library(ggplot2)
library(ascii)



Attaching package: ‘dplyr’

The following object is masked from ‘package:MASS’:

    select

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘ascii’

The following object is masked from ‘package:tidyr’:

    expand



ERROR: Error: package or namespace load failed for ‘lubridate’ in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]):
 there is no package called ‘stringr’


In [2]:
install.packages("lubridate",dependencies="TRUE")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


ERROR: Error in available[p1, dependencies, drop = FALSE]: subscript out of bounds


In [3]:
library(splines)


In [4]:
library(mgcv)

Loading required package: nlme

Attaching package: ‘nlme’

The following object is masked from ‘package:dplyr’:

    collapse

This is mgcv 1.8-23. For overview type 'help("mgcv-package")'.


In [5]:
install.packages("lubridate")

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
also installing the dependency ‘stringr’

Warning message in install.packages("lubridate"):
“installation of package ‘lubridate’ had non-zero exit status”

In [6]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [7]:
PSDS_PATH <- file.path('~')

In [9]:
lung <- read.csv(file.path(PSDS_PATH, 'LungDisease.csv'))

zhvi <- read.csv(file.path(PSDS_PATH, 'County_Zhvi_AllHomes.csv'))
zhvi <- unlist(zhvi[13,-(1:5)])

In [10]:
dates <- parse_date_time(paste(substr(names(zhvi), start=2, stop=8), "01", sep="."), "Ymd")
zhvi <- data.frame(ym=dates, zhvi_px=zhvi, row.names = NULL) %>%
  mutate(zhvi_idx=zhvi/last(zhvi))

house <- read.csv(file.path(PSDS_PATH, 'house_sales.csv'), sep='\t')

In [12]:
plot(lung$Exposure, lung$PEFR, xlab="Exposure", ylab="PEFR")
dev.off()

null device 
          1

In [13]:
model <- lm(PEFR ~ Exposure, data=lung)
model


Call:
lm(formula = PEFR ~ Exposure, data = lung)

Coefficients:
(Intercept)     Exposure  
    424.583       -4.185  


In [14]:
plot(lung$Exposure, lung$PEFR, xlab="Exposure", ylab="PEFR", ylim=c(300,450), type="n", xaxs="i")
abline(a=model$coefficients[1], b=model$coefficients[2], col="blue", lwd=2)
text(x=.3, y=model$coefficients[1], labels=expression("b"[0]),  adj=0, cex=1.5)
x <- c(7.5, 17.5)
y <- predict(model, newdata=data.frame(Exposure=x))
segments(x[1], y[2], x[2], y[2] , col="red", lwd=2, lty=2)
segments(x[1], y[1], x[1], y[2] , col="red", lwd=2, lty=2)
text(x[1], mean(y), labels=expression(Delta~Y), pos=2, cex=1.5)
text(mean(x), y[2], labels=expression(Delta~X), pos=1, cex=1.5)
text(mean(x), 400, labels=expression(b[1] == frac(Delta ~ Y, Delta ~ X)), cex=1.5)
dev.off()

null device 
          1